In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense, Add
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def residual_block(x, filters, kernel_size=3, stride=1):
    res = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    res = BatchNormalization()(res)
    res = ReLU()(res)

    res = Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(res)
    res = BatchNormalization()(res)

    if x.shape[-1] != filters or stride != 1:
        x = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(x)
        x = BatchNormalization()(x)

    x = Add()([x, res])
    x = ReLU()(x)
    return x

def create_resnet_cifar():
    input = Input(shape=(32, 32, 3))

    x = Conv2D(64, kernel_size=3, strides=1, padding='same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    num_blocks_list = [3, 3, 3]  # Number of residual blocks per stage
    filters_list = [64, 128, 256]

    for stage, (num_blocks, filters) in enumerate(zip(num_blocks_list, filters_list)):
        for block in range(num_blocks):
            stride = 2 if block == 0 and stage != 0 else 1
            x = residual_block(x, filters, stride=stride)

    x = GlobalAveragePooling2D()(x)
    output = Dense(100, activation='softmax')(x)  # 100 classes for CIFAR-100

    model = Model(inputs=input, outputs=output)
    return model

# Load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=25)

# Preprocess the data
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 100)
y_val = to_categorical(y_val, 100)
y_test = to_categorical(y_test, 100)

# Create the ResNet model for CIFAR-100
model = create_resnet_cifar()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with a specified number of epochs using the train-validation split
epochs = 50
model.fit(x_train, y_train, epochs=epochs, batch_size=100, validation_data=(x_val, y_val))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy}")


169001437/169001437 [==============================] - 12s 0us/step
Epoch 1/50
400/400 [==============================] - 65s 108ms/step - loss: 3.6138 - accuracy: 0.1525 - val_loss: 7.2874 - val_accuracy: 0.0483
Epoch 2/50
400/400 [==============================] - 41s 101ms/step - loss: 2.6984 - accuracy: 0.3037 - val_loss: 3.3055 - val_accuracy: 0.2490
Epoch 3/50
400/400 [==============================] - 41s 102ms/step - loss: 2.1712 - accuracy: 0.4191 - val_loss: 2.8682 - val_accuracy: 0.3130
Epoch 4/50
400/400 [==============================] - 43s 108ms/step - loss: 1.8173 - accuracy: 0.5001 - val_loss: 2.6162 - val_accuracy: 0.3628
Epoch 5/50
400/400 [==============================] - 41s 103ms/step - loss: 1.5420 - accuracy: 0.5614 - val_loss: 2.4938 - val_accuracy: 0.3997
Epoch 6/50
400/400 [==============================] - 44s 109ms/step - loss: 1.3140 - accuracy: 0.6196 - val_loss: 2.1425 - val_accuracy: 0.4626
Epoch 7/50
400/400 [==============================] - 42s 104m